# Project Name : Bubble sheet scanner and test grader using OMR, Python, and OpenCV

# Optical Mark Recognition (OMR)

Optical Mark Recognition, or OMR for short, is the process of automatically analyzing human-marked documents and interpreting their results.

# Implemention Algortithm

1: Detecting the exam in an image.

2: Applying a perspective transform to extract the top-down, birds-eye-view of the exam.

3: Extracting the set of bubbles (i.e., the possible answer choices) from the perspective transformed exam.

4: Sorting the questions/bubbles into rows.

5: Determining the marked (i.e., “bubbled in”) answer for each row.

6: Looking up the correct answer in our answer key to determine if the user was correct in their choice.

7: Repeating for all questions in the exam.

# Perspective Transformation 

In Perspective Transformation, , we can change the perspective of a given image or video for getting better insights about the required information. In Perspective Transformation, we need provide the points on the image from which want to gather information by changing the perspective. We also need to provide the points inside which we want to display our image. Then, we get the perspective transform from the two given set of points and wrap it with the original image.

# Code

In [ ]:
# importing the necessary packages

from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2 

In [ ]:
# defining the answer key which maps the question number
# to the correct answer

# (Q.1 - B ,Q.2 - E, Q.3 - A, Q.4 - D, Q.5 - B)

ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

In [ ]:
# loading the image, converting it to grayscale, blurring it
# slightly, then finding edges

image = cv2.imread("test_01.png")
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(blurred,75,200)
# cv2.imshow("Output",image)
# cv2.waitKey(0)

In [ ]:
# finding contours in the edge map, then initializing
# the contour that corresponds to the document

cnts = cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,
                       cv2.CHAIN_APPROX_SIMPLE)

cnts = imutils.grab_contours(cnts)

docCnt = None

# at least one contour was found
# sorting the contours according to their size in
# descending order

if len(cnts)>0:
    cnts = sorted(cnts , key = cv2.contourArea , reverse = True)

#looping over the contours
# then approximate the contour

    for c in cnts :
        peri = cv2.arcLength(c,True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        
        if len(approx) == 4 :
            docCnt = approx
            break
        

In [ ]:
# applying a four point perspective transform to both the
# original image and grayscale image to obtain a top-down
# birds eye view of the paper


paper = four_point_transform(image,docCnt.reshape(4,2))
warped = four_point_transform(gray,docCnt.reshape(4,2))

In [ ]:
# cv2.imshow("Output",warped)
# cv2.waitKey(0)

# cv2.imshow("Output",warped)
# cv2.waitKey(0)

In [ ]:
# applying Otsu's thresholding method to binarize the warped
# piece of paper

thresh = cv2.threshold(warped,0,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# cv2.imshow("Output",thresh)
# cv2.waitKey(0)


In [ ]:
def test_grader(image):
    
    # finding contours in the thresholded image, then initializing
    # the list of contours that correspond to questions
    
    image = cv2.imread(image)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    questionCnts = []

    # looping over the contours

    for c in cnts:
        # computing the bounding box of the contour, then using the
        # bounding box to derive the aspect ratio

        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)

        # in order to label the contour as a question, region
        # should be sufficiently wide, sufficiently tall, and
        # have an aspect ratio approximately equal to 1

        if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
            questionCnts.append(c)

    # sorting the question contours top-to-bottom, then initialize
    # the total number of correct answers

    questionCnts = contours.sort_contours(questionCnts,
    method="top-to-bottom")[0]
    correct = 0

    # each question has 5 possible answers, to loop over the
    # question in batches of 5

    for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):

        # sorting the contours for the current question from
        # left to right, then initialize the index of the
        # bubbled answer

        cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
        bubbled = None

        # loop over the sorted contours
        for (j, c) in enumerate(cnts):
            # constructing a mask that reveals only the current
            # "bubble" for the question

            mask = np.zeros(thresh.shape, dtype="uint8")
            cv2.drawContours(mask, [c], -1, 255, -1)

            # applying the mask to the thresholded image, then
            # count the number of non-zero pixels in the
            # bubble area

            mask = cv2.bitwise_and(thresh, thresh, mask=mask)
            total = cv2.countNonZero(mask)

            # if the current total has a larger number of total
            # non-zero pixels, then we are examining the currently
            # bubbled-in answer

            if bubbled is None or total > bubbled[0]:
                bubbled = (total, j)

        # initializing the contour color and the index of the
        # *correct* answer

        color = (0, 0, 255)
        k = ANSWER_KEY[q]

        # check to see if the bubbled answer is correct
        if k == bubbled[1]:
            color = (0, 255, 0)
            correct += 1

        # draw the outline of the correct answer on the test
        cv2.drawContours(paper, [cnts[k]], -1, color, 3)



In [ ]:
# Calling the function

test_grader("test_01.png")

score = (correct / 5.0) * 100
print("[INFO] Obtained score: {:.2f}%".format(score))

cv2.putText(paper, "Score:{:.2f}%".format(score), (10, 30),
cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)